In [0]:
import json
from pathlib import Path

# Variables
job_settings                = json.loads(dbutils.widgets.get("job_settings"))
table                       = job_settings['table']
settings                    = json.loads(next(Path().glob(f"./layer_*_bronze/{table}.json")).read_text())
dst_table_name              = settings.get("dst_table_name")
badRecordsPath              = settings.get("readStreamOptions", {}).get("badRecordsPath")

try:
    dbutils.fs.ls(badRecordsPath)
    df = spark.read.json(badRecordsPath)
    df.write.mode("overwrite").format("delta").saveAsTable(f"{dst_table_name}_bad_records")
except:
    spark.sql(f"DROP TABLE IF EXISTS {dst_table_name}_bad_records")

if spark.catalog.tableExists(f"{dst_table_name}_bad_records"):
    raise Exception(f"Bad records table exists: {dst_table_name}_bad_records")